# __Dynamic Response of a Multi-Story building with _`COMSOL Multiphysics`_ __
## __Hands on__

GRAPHS WITHOUT LEGENDS AND UNITS WILL NOT BE CONSIDERED!!

<img style='width: 700px; margin: 0px; float: left; border: 0px solid black;' src="images/comsol_building.png">

#### Setup

In [ ]:
# Load fundamental modules
from math import sin
import numpy as np
import pandas as pd
from numpy.linalg import inv, norm
from scipy.linalg import eigh
from scipy.signal import find_peaks
from matplotlib import pyplot as plt

In [ ]:
# Set up properties
dofs = 3 # number of dofs
Lx = 5 # Floor x-side [m]
Ly = 5 # Floor y-side [m]
ft = 0.3 # Floor thickness [m]

Hf = 3 # Floor's height [m]
bc = 0.3 # Cross sectional base for column elements [m]
hc = 0.3 # Cross sectional height for column elements [m]

E_floor = 25e9 # Floor concrete Young's modulus [Pa]
E_beam = 28.3e9 # Beam/Column concrete Young's modulus [Pa]

rho_floor = 2300 # Floor concrete unit mass density [kg/m^3]
rho_beam = 500 # Beam/Column concrete unit mass density [kg/m^3]

# time vector
dt = 1.0e-2 # time step
dur = 60.0 # duration
vtm = np.arange(0.0,dur,dt) # time vector
ntm = vtm.size # number of time steps

# Compute the transfer function using the Fast Fourier Transform (use np.fft.rfft - https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html)
def next_power_of_2(x):  
    return 1 if x == 0 else 2**(x - 1).bit_length()

# earthquake input acceleration (inertial force)  [time (s),acceleration (m/s/s)]
eqk = np.loadtxt("seisme.txt")
aeq = lambda s: np.interp(s,eqk[:,0],eqk[:,1])
plt.ion()
plt.plot(vtm,aeq(vtm),label=r"$a_{ext}$")
plt.xlim(0.0,dur)
plt.xlabel(r'$t [s]$')
plt.ylabel(r'$a [m/s/s]$')
plt.legend(frameon=False)
plt.ioff()
plt.show()

#### __[TASK #1] _`COMSOL Multiphysic`_ 3D Modal Shapes and Eigen-frequencies__ [1/20 points]

In the `Global Definitions`, select `Nx`=`Ny`=1 and `Nf`=3, with no missing frames
(`Mx`=`My`=`Mf`=0). 

Rebuild the `Geometry` by building all the entities (in `Form Union (fin)`)

* Compute the `Eigenfrequencies` study for 10 Eigen-frequencies (real) around 1 Hz and
report the corresponding `Modal Shapes` here (snapshots and frequency values to be placed in the cell Report the Modal Shapes Eigen-frequencies from _`COMSOL Multiphysics`_)

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn1.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn2.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn3.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn4.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn5.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn6.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn8.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn9.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn10.png">

* Evaluate and export `umx`, `umy` and `umz` from _`COMSOL Multiphysics`_ and plot them in the notebook. `umx`, `umy` and `umz` are the floor's midsurface displacements (one at each floor's corner) in the $x$, $y$ and $z$ direction.

In [ ]:
comsol_eigs3D = ...

print(comsol_eigs3D.columns)

plt.ion()
for i in range(10):
    plt.plot(...,np.array([0,1,2,3]),'--s',label=r'$\phi_{:>d}$'.format(i))

plt.legend(frameon=False)
plt.ioff()

* Evaluate and export the Participation Factors $\Gamma_{ij}$ from _`COMSOL Multiphysics`_ (*Hint*: Export Group `Participation Factors (Eigenfrequencies)`).

*Nota bene*: Using modal participation factors is a way to characterize how much a certain mode will be excited by a rigid body acceleration $\boldsymbol{d}_j$ in a certain direction $j$. The _normalized participation factor_ with respect to mode $i$ and excitation direction $j$, is defined as:

$$\tilde{\Gamma}_{ij}=\frac{\left<\boldsymbol{\phi}_i,\boldsymbol{M}.\boldsymbol{d}_j\right>}{\left<\boldsymbol{\phi}_i,\boldsymbol{M}.\boldsymbol{\phi}_j\right>}=\frac{\Gamma_{ij}}{m_i}$$

with $\Gamma_{ij}=\left<\boldsymbol{\phi}_i,\boldsymbol{M}.\boldsymbol{d}_j\right>$ the Participation Factors and $m_i=\left<\boldsymbol{\phi}_i,\boldsymbol{M}.\boldsymbol{\phi}_j\right>$ the $i^{\text{th}}$ modal mass.

In [ ]:
comsol_Gamma3D = ...
print(np.real(comsol_Gamma3D))

* Evaluate and export the _effective modal masses_ $m_{\text{eff},ij}$ and compare them with the _`COMSOL Multiphysics`_ ones

*Nota bene*: in _`COMSOL Multiphysics`_  the `Effective Participation Mass (Eigenfrequencies)` are computed (in the `Export Group`). The _effective modal mass_ is a quantity related to the modal participation factor. The effective modal mass for mode $i$, with respect to excitation in direction $j$, is defined from the participation factor and the modal mass as:

$$m_{\text{eff},ij}=m_i\cdot \Gamma_{ij}^2$$

In [ ]:
comsol_masses3D = ...
print(np.real(comsol_masses3D))

#### __[TASK #2] Story Shear Building Model (1D translation along $x$)__ [3/20 points]

Verify the consistency of the 3D FEM _`COMSOL Multiphysics`_ model with the simplified 3DOF system.

Constrain the structure in the $x$-$z$ plane. For each physical entity `Floors`, `Columns`, `Beams`, enable the adequate kinematic constraints. Constrain `Columns` to pure shear.

<img style='width: 700px; margin: 0px; float: left; border: 0px solid black;' src="images/double_clamped_beam.png">

* Compute the `Eigenfrequencies` study for 10 Eigen-frequencies (real) around 1 Hz and
report the corresponding `Modal Shapes` here (snapshots and frequency values to be placed in the cell Report the Modal Shapes Eigen-frequencies from _`COMSOL Multiphysics`_)

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn1ssb.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn2ssb.png">

<img style='width: 500px; margin: 0px; float: left; border: 0px solid black;' src="fn3ssb.png">

* Compute the approximate mass and stiffness matrices of each story of the _`COMSOL Multiphysics`_ model

_Nota Bene_: do not consider the masses of the beams and columns (double clamped beam)

In [ ]:
k = ... # Columns' global stiffness
m = ... # Floor mass [kg]

# Mass matrix
m1 = m
m2 = m
m3 = m
M = np.array([[m1,0,0],[0,m2,0],[0,0,m3]])

# Stiffness matrix
k1 = k
k2 = k
k3 = k
K = np.array([[k1+k2,-k2,0],[-k2,k2+k3,-k3],[0,-k3,k3]])

* Compute the eigen-frequencies and  eigen-modes of the MDOF system. What are the natural frequencies? Do they correspond to the _`COMSOL Multiphysics`_ ones?

Answer:

In [ ]:
# find natural frequencies and mode shapes (https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html)
evals, evecs = ...
frequencies = ...
print('f1 = {0} Hz, f2 = {1} Hz, f3 = {2} Hz '.format(*tuple(frequencies.round(2).tolist())))
print('l1 = {0}, l2 = {1}, l3 = {2} '.format(*tuple(evecs.round(2).tolist())))

plt.ion()
plt.plot(np.concatenate([np.array([0]),evecs[:,0]]),np.array([0,1,2,3]),'--o',label=r'$\phi_1$')
plt.plot(np.concatenate([np.array([0]),evecs[:,1]]),np.array([0,1,2,3]),'--o',label=r'$\phi_2$')
plt.plot(np.concatenate([np.array([0]),evecs[:,2]]),np.array([0,1,2,3]),'--o',label=r'$\phi_3$')
plt.legend(frameon=False)
plt.ioff()

* Plot the eigen-modes of MDOF system and compare them to the _`COMSOL Multiphysics`_ ones (evaluated on at the midsurface of each floor)

_Hint_: Export `umx`, `umy` and `umz` from _`COMSOL Multiphysics`_ and import the files in the notebook.

Answer:

In [ ]:
comsol_eigs = ...
print(np.real(comsol_eigs))

In [ ]:
plt.ion()
for i in range(10):
    plt.plot(...,np.array([0,1,2,3]),'--s',label=r'$\phi_{:>d}$'.format(i))

plt.legend(frameon=False)
plt.ioff()

* Compute the Participation Factors $\Gamma_{ij}$ and compare them to those from _`COMSOL Multiphysics`_ ((from Export Groups `Participation Factors (Eigenfrequencies)`).

In [ ]:
# Analytical Compute modal Masses
mM = ...
print("Nodal mass 1: {:>.2f} kg".format(mM[0]))
print("Nodal mass 2: {:>.2f} kg".format(mM[1]))
print("Nodal mass 3: {:>.2f} kg".format(mM[2]))

# Analytical Participation Factors
Gamma_tilde = ...

print(Gamma_tilde.sum(axis=-1))

In [ ]:
comsol_Gamma = ...
print(np.real(comsol_Gamma))

* Compute the _effective modal masses_ $m_{\text{eff},ij}$ and compare them with the _`COMSOL Multiphysics`_ ones

*Nota bene*: the `scipy` function `eigh` normalizes the eigenvector as:

$$              \boldsymbol{K} . \boldsymbol{\phi}_n = ω^2 \cdot \boldsymbol{M} . \boldsymbol{\phi}_n$$

The corresponding `python` code:

```
              K @ vi = λ * M @ vi
```
with:

$$\lambda = \omega_i^2$$


$$\boldsymbol{\phi}_n^T . \boldsymbol{K} . \boldsymbol{\phi}_n = \omega_n^2$$
$$\boldsymbol{\phi}_n^T . \boldsymbol{M} . \boldsymbol{\phi}_n = 1$$

In [ ]:
mMeff = ...
print(mMeff)

In [ ]:
comsol_masses = ...
print(np.real(comsol_masses))

#### __[TASK #3] _`COMSOL Multiphysic`_  3D Response to Earthquake loading__ [4/20 points]

Consider the original 3D building (disable the kinematica constraints for Simple Shear Building).

* Solve the SDOF problem for each eigen-mode, in order to find the modal amplitudes $\alpha_n(t)$ (consider a critical damping $\xi$=2%). Construct the external force for the seismic load induced by $\boldsymbol{a}_{eq} = a_{eq}(t)\begin{bmatrix} 1\\ 1\\ 1\end{bmatrix} = a_{eq}(t)\sum_{i=1}^3 \boldsymbol{d}_i$ using the _`COMSOL Multiphysics`_ participation factors $\Gamma_{ij}$


_Nota Bene_:

The second order equation $y''(t)= a y'(t)+ b y(t) + c(t)$ can be rewritten as a system:

(1)   $y'(t) = z(t)$

(2)   $z'(t) = a z(t) + b y(t) + c(t)$

In this case, $a=0$ and $b=-\boldsymbol{M^{-1}.K}$.

The final first-order ordinary differential system of equations reads: 

$$\boldsymbol{X}'(t)=\boldsymbol{A.X(t)}+\boldsymbol{b}(t)$$

with $X(t)=\begin{bmatrix} \boldsymbol{q}(t)\\ \boldsymbol{q}'(t) \end{bmatrix}$ and $A=\begin{bmatrix} \boldsymbol{0} & \boldsymbol{I} \\  -\boldsymbol{M^{-1}.K} & \boldsymbol{0} \end{bmatrix}$ and with $\boldsymbol{b}(t)=\begin{bmatrix}0 \\ c(t)\end{bmatrix}$

In [ ]:
# Hint: use the scipy function solve_ivp (https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)
from scipy.integrate import solve_ivp

# Initial conditions
y0 = np.zeros((1,))
z0 = np.zeros((1,))
X0 = np.concatenate([y0,z0])

def solve2ndOrderODE(t,X,n,xi):
    # with extra arguments args:
    # n = number of mode
    # xi = damping coefficient
    ω_n = ... # n-th natural pulsation
    Σj_Γ_nj = ...
    fn = ... # project external force on the n-th eigen mode
    return np.array([X[1],...]) # return [y(t),z(t)]

In [ ]:
alpha1 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(0,0.02))


plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha1.y[0,:])
axs[0].legend([r'$q_1$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha1.y[1,:])
axs[1].legend([r'$\dot{q}_1$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha2 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(1,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha2.y[0,:])
axs[0].legend([r'$q_2$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha2.y[1,:])
axs[1].legend([r'$\dot{q}_2$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha3 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(2,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha3.y[0,:])
axs[0].legend([r'$q_3$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha3.y[1,:])
axs[1].legend([r'$\dot{q}_3$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha4 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(3,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha4.y[0,:])
axs[0].legend([r'$q_4$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha4.y[1,:])
axs[1].legend([r'$\dot{q}_4$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha5 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(4,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha5.y[0,:])
axs[0].legend([r'$q_5$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha5.y[1,:])
axs[1].legend([r'$\dot{q}_5$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha6 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(5,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha6.y[0,:])
axs[0].legend([r'$q_6$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha6.y[1,:])
axs[1].legend([r'$\dot{q}_6$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha7 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(6,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha7.y[0,:])
axs[0].legend([r'$q_7$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha7.y[1,:])
axs[1].legend([r'$\dot{q}_7$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha8 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(7,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha8.y[0,:])
axs[0].legend([r'$q_8$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha8.y[1,:])
axs[1].legend([r'$\dot{q}_8$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha9 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(8,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha9.y[0,:])
axs[0].legend([r'$q_9$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha9.y[1,:])
axs[1].legend([r'$\dot{q}_9$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

In [ ]:
alpha10 = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(9,0.02))

plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(alpha1.t,alpha10.y[0,:])
axs[0].legend([r'$q_{10}$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$q(t) [m]$')

axs[1].plot(alpha1.t,alpha10.y[1,:])
axs[1].legend([r'$\dot{q}_{10}$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{q}(t) [m/s]$')

plt.ioff()

* Compute the total response based on the $\alpha_n(t)$ ahd $\boldsymbol{\phi}_n$ values

Answer:

In [ ]:
# Modal amplitudes (displacement)
Alpha = np.array([alpha1.y[0,:],alpha2.y[0,:],alpha3.y[0,:],
                  alpha4.y[0,:],alpha5.y[0,:],alpha6.y[0,:],
                  alpha7.y[0,:],alpha8.y[0,:],alpha9.y[0,:],
                 alpha10.y[0,:]])

# Displacement
u = ...


# Modal amplitudes (velocity)
AlphaDot = np.array([alpha1.y[1,:],alpha2.y[1,:],alpha3.y[1,:],
                     alpha4.y[1,:],alpha5.y[1,:],alpha6.y[1,:],
                     alpha7.y[1,:],alpha8.y[1,:],alpha9.y[1,:],
                    alpha10.y[1,:]])


# Velocity
v = ...

In [ ]:
plt.ion()

fig,axs = plt.subplots(3,1,sharex=True,sharey=True)
axs[0].plot(alpha1.t,u[0,:])
axs[0].legend([r'$u_1$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$u_1(t) [m]$')

axs[1].plot(alpha2.t,u[1,:])
axs[1].legend([r'$u_2$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$u_2(t) [m]$')

axs[2].plot(alpha3.t,u[2,:])
axs[2].legend([r'$u_3$',],frameon=False)
axs[2].set_xlabel(r'$t [s]$')
axs[2].set_ylabel(r'$u_3(t) [m]$')

plt.ioff()

np.savetxt('q1.csv',np.hstack([vtm.reshape(-1,1),u[0,:].reshape(-1,1)]),delimiter = ',', fmt='%15.5f')
np.savetxt('q2.csv',np.hstack([vtm.reshape(-1,1),u[1,:].reshape(-1,1)]),delimiter = ',', fmt='%15.5f')
np.savetxt('q3.csv',np.hstack([vtm.reshape(-1,1),u[2,:].reshape(-1,1)]),delimiter = ',', fmt='%15.5f')

In [ ]:
plt.ion()

fig,axs = plt.subplots(3,1,sharex=True,sharey=True)
axs[0].plot(alpha1.t,v[0,:])
axs[0].legend([r'$v_1$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$v_1(t) [m/s]$')

axs[1].plot(alpha2.t,v[1,:])
axs[1].legend([r'$v_2$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$v_2(t) [m/s]$')

axs[2].plot(alpha3.t,v[2,:])
axs[2].legend([r'$v_3$',],frameon=False)
axs[2].set_xlabel(r'$t [s]$')
axs[2].set_ylabel(r'$v_3(t) [m/s]$')

plt.ioff()

* Compute and plot the _response spectrum_ for each modal amplitude $\alpha_i$:

$$S_d(\alpha_i)=\max_{t}\vert \alpha_i(t)\vert$$

and plot the values as a function of the associated natural frequency.

Answer:

In [ ]:
Sd = ...

plt.ion()
col = np.random.rand(20,3)
# tick_params(which='both', width=2)
for i in range(10):
    plt.vlines(np.real(comsol_eigs3D.iloc[i,0]),0.0,50.0,label=r'$f_{{{:>d}}} = {:>.1f}\quadHz$'.format(i,np.real(comsol_eigs3D.iloc[i,0])),
              color=col[2*i:2*i+1,:])

plt.plot(np.real(comsol_eigs3D.iloc[:,0].values),Sd,'--o',color='black')
plt.xlabel(r'$f [Hz]$')
plt.ylabel(r'$Sd [m]$')
plt.xscale('log')
plt.yscale('log')
plt.xlim(0.1,50)
plt.grid(True, which='both')
plt.legend(frameon=False)
plt.ioff()

Which mode dominates the transient response?

Answer:

The three translation modes

#### __[TASK #4] Compute transfer function__  [4/20 points]

For each mode, the transfer function $\hat{H}_n(\omega)$ reads:

$$\hat{\boldsymbol{H}}(\omega)=\sum_n H_n(\omega)\boldsymbol{\phi}_n\otimes\boldsymbol{\phi}_n$$

* Compute the transfer function of the whole MDOF (consider a critical damping $\xi$=2%)

Answer: 

In [ ]:
def computeTF(sol,n,xi):
    
    ω_n = ... # n-th natural pulsation
    Σj_Γ_nj = ...
    fn = ... # project external force on the n-th eigen mode
    fn = np.vectorize(fn)
    
    nfft = next_power_of_2(vtm.size)
    dfq = ...
    vfq = ...
    Inp = ... # real fft
    Out = ... # real fft
    
    return vfq,Out/(Inp+1.0e-4)

vfq,H1 = computeTF(alpha1,0,0.02)
_,H2 = computeTF(alpha2,1,0.02)
_,H3 = computeTF(alpha3,2,0.02)
_,H4 = computeTF(alpha1,3,0.02)
_,H5 = computeTF(alpha2,4,0.02)
_,H6 = computeTF(alpha3,5,0.02)
_,H7 = computeTF(alpha1,6,0.02)
_,H8 = computeTF(alpha2,7,0.02)
_,H9 = computeTF(alpha3,8,0.02)
_,H10 = computeTF(alpha1,9,0.02)

# eigen vectors
evecs = ...
# norm of eigen vectors
normevecs = np.linalg.inv(np.diag(np.linalg.norm(evecs,axis=0)))

# Transfer function H
H = np.einsum('ik,jk,kl->ijl',...)

# Fourier's spectrum H1*phi1+H2*phi2+H3*phi3
U = np.einsum('jk,kl->jl',...)

* Plot the response `U` and the eigen frequencies previously identified. What do you observe?

Answer:

In [ ]:
plt.ion
plt.loglog(vfq,np.abs(U.T))
plt.vlines(np.real(comsol_eigs3D.iloc[0,0]),0.1,1.0,color='blue',linestyle='--')
plt.vlines(np.real(comsol_eigs3D.iloc[1,0]),0.1,1.0,color='orange',linestyle='--')
plt.vlines(np.real(comsol_eigs3D.iloc[2,0]),0.1,1.0,color='green',linestyle='--')
plt.xlabel(r'$f [Hz]$')
plt.xlim([0.1,10.0])
plt.ylim([0.1,10.0])
plt.ylabel(r'$\vert TF\vert [1]$')
plt.legend([r'$u_{{{:d}}}(f)$'.format(m) for m in range(3)]+[r'$f_{{{:d}}}$'.format(m) for m in range(3)],frameon=False,loc='upper left')
plt.ioff()

Explanation: the peaks correspond to the eigen frequencies. Amplitudes are different. The first mode is dominant

#### __[TASK #5] Response to a random load (wind)__  [7/20 points]

We study the main characteristics of the wind loading. The horizontal velocity of the wind is modeled here in the following way:

$$V(t) = \mu_V+\beta(t)$$

where $\beta(t)$ is a centered gaussian random process, with a given Power Spectral Density $S_V(f)$ (with $f$ being the frequency, in Hz), and $\mu_V$ = 100 km/h. The velocity field is therefore random and $S_V(f)$ reads:

$$ S_V(f) = \frac{4\kappa\cdot \mu_V\cdot L}{\left(2+\left(\frac{L\cdot f}{\mu_V} \right)^2\right)^{\frac{5}{6}} }$$

with $\kappa$ and $L$ two constants.

* Plot the spectrum $S_V(f)$ as a function of $f$ (both phase and amplitude!). What are the mathematical properties of this Power Spectral Density?

_Nota Bene_: use the frequency vector employed in the previous steps, considering the time vector `vtm` as time domain of reference.

Answer:

In [ ]:
muV = 1.0e5/3600.0 # mean velocity at 10m height: 100 km/h
kappa = 0.002 # constant
Lcar = 1200.0 # constant

nfft = next_power_of_2(vtm.size)
dfq = ...
vfq = ...
    
Sv = ...

plt.ion()
plt.loglog(vfq,Sv(vfq),color='black')
plt.xlim(0.01,10.0)
plt.xlabel(r'$f [Hz]$')
plt.ylabel(r'$S_V(f) [m^2]$')
plt.ioff()

In order to generate a plausible realization of the wind velocity field $\beta_j(t)$, the following formula is adopted:

$$ \beta_j(t) = 2\cdot\sqrt{\frac{2}{\pi}}\cdot \sum_{k=1}^{N_f} \sqrt{2\cdot\pi\cdot S_V(f_k)\cdot \Delta f }\cdot \rho_{jk} \cdot\cos\left( 2\pi f_k t -2\pi\phi_{jk}\right)$$

with:
 - $\Delta f$ : frequency step
 - $N_f$: number of frequency points 
 - $\rho_{jk}=\vert a_{jk}+i\cdot b_{jk} \vert$: $j^{\text{th}}$-realization of the random vector $\boldsymbol{\rho}$ with $a_{jk}\sim\mathcal{N}(0,1)$ and $b_{jk}\sim\mathcal{N}(0,1)$
 - $\phi_{jk}$: $j^{\text{th}}$-realization of the random vector $\boldsymbol{\phi}$ with $\phi_{jk}\sim\mathcal{U}(0,1)$

* Generate $N_s$ samples of wind velocity field

Answer:

In [ ]:
# Hint: Use the numpy random number generator https://numpy.org/doc/stable/reference/random/index.html
from numpy.random import default_rng
rng = default_rng()

Ns = 10 # number of wind samples

def beta_i(t):
    nfft = next_power_of_2(vtm.size)
    dfq = ...
    vfq = ...
    vwo = ...
    
    # random amplitude (Normal distribution)
    a = ...
    b = ...
    rho = ...

    # random phase (Uniform distribution)
    phi = ...

    return ...

def beta(t,Ns):
    all_beta=np.empty((Ns,vtm.size))
    for i in range(Ns):
        all_beta[i,:]=beta_i(t)
    return all_beta

beta_realizations = beta(vtm,Ns)
Ebeta = np.mean(beta_realizations,0)

* Plot the $N_s$ realizations on a single graph

Answer:

In [ ]:
plt.ion()
plt.plot(vtm,beta_realizations.T)
plt.xlim(0.0,dur)
plt.xlabel(r'$t [s]$')
plt.ylabel(r'$\beta [m/s]$')
plt.ioff()

* Compute the Fourier's transform of each realization and plot them on a single graph

Answer:

In [ ]:
nfft = next_power_of_2(vtm.size)
dfq = ...
vfq = ...
beta_fs = np.fft.rfft(beta_realizations,nfft,axis=-1)*dt # real fft


plt.ion()
plt.loglog(vfq,np.abs(beta_fs.T))
plt.xlabel(r'$f [Hz]$')
plt.ylabel(r'$\vert\mathcal{F}(\beta)\vert [m]$')
plt.ioff()

* Compute the approximated and the theoretical Power Spectral Density for the velocity field $V(t)$

In [ ]:
# velocity realizations
v_realizations = muV+beta_realizations
# empirical velocity average
Ev = ...
v_fs = np.fft.rfft(v_realizations-Ev,nfft,axis=-1)*dt # real fft
Sv_a = ... # approximated PSD

plt.ion()
plt.loglog(vfq,Sv_a.T,color='lightgrey',linewidth=0.1)
plt.loglog(vfq,np.mean(Sv_a,0),label=r'$S_v^{ap}$',color='grey')
plt.loglog(vfq,Sv(vfq),label=r'$S_v^{th}$',color='black',linestyle='--')
plt.legend(frameon=False)
plt.xlabel(r'$f [Hz]$')
plt.ylabel(r'$S(f) [m^2/s]$')
plt.ioff()

The pressure $p$ applied by the wind on a given surface can be related to the square of the velocity through the formula:

$$ p(t)=\frac{1}{2}\rho_aC_D V^2(t)$$

with $\rho_a$=1.2 kg/m$^3$ being the unit mass density of the air and $C_D$ the drag coefficient.

In [ ]:
rho_a = 1.2 # kg/m**3
Cd = 0.8

However, in practice a linearized formula for the pressure field is adopted hereafter.

* Compute the linearized expression of the pressure field, considering small fluctuations $\beta(t)\ll 1$

Answer:

$$p(t)=\frac{1}{2}\rho_a C_D\mu_V^2\left(1+2\frac{\beta(t)}{\mu_V}\right)$$

* Compute several samples of the pressure field for $z$=1 m, evaluate its power spectrum $S_p(f)$ and compare them with the theoretical spectrum:

$$S_p(f)=4\left(\frac{\mu_p}{\mu_V}\right)^2 S_V(f)$$

where $\mu_p$ is the mean pressure.

Answer:

In [ ]:
# Mean pressure field
mup = ...

# theoretical PSD
Sp_t = ...

# pressure field
p_realizations = ...
# empirical pressure average
Ep = ...

p_fs = np.fft.rfft(p_realizations-Ep,nfft,axis=-1)*dt # real fft
Sp_a = ...


plt.ion()
plt.loglog(vfq,Sp_a.T,color='lightgrey',linewidth=0.5)
plt.loglog(vfq,np.mean(Sp_a,0),label=r'$S_p^{ap}$',color='grey',linewidth=0.5)
plt.loglog(vfq,Sp_t(vfq),label=r'$S_p^{th}$',color='black',linestyle='--')
plt.legend(frameon=False)
plt.xlabel(r'$f [Hz]$')
plt.ylabel(r'$S(f) [N^2s/m^4]$')
plt.ioff()

Adopting the _`COMSOL Multiphyiscs`_ 3D building, consider the same pressure field $p(t)$ on each translational dof, using the previously computed _participation factors_ and with a lateral surface (where the wind is applied) equal to $S$=4 m$^2$

* Compute the amplitudes $\alpha_n$ for for all the $N_S$ random wind load realizations 

_Nota bene_: use `solve2ndOrderODE`!

Answer:

In [ ]:
# Initial conditions
y0 = np.zeros((1,))
z0 = np.zeros((1,))
X0 = np.concatenate([y0,z0])
S= 4.0 # lateral surface area [m**2]

def solve2ndOrderODE(t,X,n,xi,i):
    # with extra arguments args:
    # n = number of mode
    # xi = damping coefficient
    ω_n = ... # n-th natural pulsation
    Σj_Γ_nj = ...
    
    pwind = lambda s: ...
    fn = lambda s: ... # project external force on the n-th eigen mode
    
    return np.array([X[1],...)])

AlphaRndm = np.empty((Ns,3,ntm))
AlphaDotRndm = np.empty((Ns,3,ntm))
uRndm = np.empty((Ns,3,ntm))
vRndm = np.empty((Ns,3,ntm))

# important modes : 0, 3, 5 (translational in x direction)
for r in range(Ns):
    alpha1Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(0,0.02,r))
#     alpha2Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(1,0.02,r))
#     alpha3Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(2,0.02,r))
    alpha4Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(3,0.02,r))
#     alpha5Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(4,0.02,r))
    alpha6Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(5,0.02,r))
#     alpha7Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(6,0.02,r))
#     alpha8Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(7,0.02,r))
#     alpha9Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(8,0.02,r))
#     alpha10Rndm = solve_ivp(solve2ndOrderODE, (0.0, dur), X0, t_eval=vtm, args=(9,0.02,r))
    
    # Modal amplitudes (displacement)
    AlphaRndm[r,:,:] = np.vstack([alpha1Rndm.y[0,:],alpha4Rndm.y[0,:],alpha6Rndm.y[0,:]])

    # Displacement
    uRndm[r,:,:] = ...

    # Modal amplitudes (velocity)
    AlphaDotRndm[r,:,:] = np.vstack([alpha1Rndm.y[1,:],alpha2Rndm.y[1,:],alpha3Rndm.y[1,:]])

    # Velocity
    vRndm[r,:,:] = ...

In [ ]:
plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(vtm,AlphaRndm[:,0,:].squeeze().T)
axs[0].legend([r'$\alpha_1$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$\alpha_1(t) [m]$')
# axs[0].set_ylim(-600.0,600.0)

axs[1].plot(vtm,AlphaDotRndm[:,0,:].squeeze().T)
axs[1].legend([r'$\dot{\alpha}_1$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{\alpha}_1(t) [m/s]$')
# axs[1].set_ylim(-5500.0,5500.0)

plt.ioff()

In [ ]:
plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(vtm,AlphaRndm[:,1,:].squeeze().T)
axs[0].legend([r'$\alpha_2$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$\alpha_2(t) [m]$')
# axs[0].set_ylim(-600.0,600.0)

axs[1].plot(vtm,AlphaDotRndm[:,1,:].squeeze().T)
axs[1].legend([r'$\dot{\alpha}_2$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{\alpha}_2(t) [m/s]$')
# axs[1].set_ylim(-5500.0,5500.0)

plt.ioff()

In [ ]:
plt.ion()

fig,axs = plt.subplots(2,1,sharex=True)
axs[0].plot(vtm,AlphaRndm[:,2,:].squeeze().T)
axs[0].legend([r'$\alpha_3$',],frameon=False)
axs[0].set_xlabel(r'$t [s]$')
axs[0].set_ylabel(r'$\alpha_3(t) [m]$')
# axs[0].set_ylim(-600.0,600.0)

axs[1].plot(vtm,AlphaDotRndm[:,2,:].squeeze().T)
axs[1].legend([r'$\dot{\alpha}_3$',],frameon=False)
axs[1].set_xlabel(r'$t [s]$')
axs[1].set_ylabel(r'$\dot{\alpha}_3(t) [m/s]$')
# axs[1].set_ylim(-5500.0,5500.0)

plt.ioff()

* Plot the average Power Spectral Density $\boldsymbol{S}_u$ for the each dof and the theoretical one (use the theoretical $FRF$ computed above!)

Answer:

In [ ]:
# Conjugate transpose Transfer function
HcT = np.transpose(np.conjugate(H),(1,0,2))

def compute_Su(fq,Sf):
    fq_idx = np.argwhere(np.isin(vfq,fq))

    Su = np.empty((3,3,fq.size),dtype=np.complex64)
    for i in fq_idx:
        Su[:,:,i] = ...
    return np.real(Su)

Su_a = compute_Su(vfq,Sf_a)
Su_t = compute_Su(vfq,Sf_t(vfq))

In [ ]:
plt.ion()
fig,axs = plt.subplots(3,3,sharex=True,sharey=True)
axs[0,0].loglog(vfq,Su_a[0,0,:],label=r'$S_{u1-u1}^{ap}$')
axs[0,1].loglog(vfq,Su_a[0,1,:],label=r'$S_{u1-u2}^{ap}$')
axs[0,2].loglog(vfq,Su_a[0,2,:],label=r'$S_{u1-u3}^{ap}$')
axs[1,0].loglog(vfq,Su_a[1,0,:],label=r'$S_{u2-u1}^{ap}$')
axs[1,1].loglog(vfq,Su_a[1,1,:],label=r'$S_{u2-u2}^{ap}$')
axs[1,2].loglog(vfq,Su_a[1,2,:],label=r'$S_{u2-u3}^{ap}$')
axs[2,0].loglog(vfq,Su_a[2,0,:],label=r'$S_{u3-u1}^{ap}$')
axs[2,1].loglog(vfq,Su_a[2,1,:],label=r'$S_{u3-u2}^{ap}$')
axs[2,2].loglog(vfq,Su_a[2,2,:],label=r'$S_{u3-u3}^{ap}$')

axs[0,0].loglog(vfq,Su_t[0,0,:],label=r'$S_{u1-u1}^{th}$')
axs[0,1].loglog(vfq,Su_t[0,1,:],label=r'$S_{u1-u2}^{th}$')
axs[0,2].loglog(vfq,Su_t[0,2,:],label=r'$S_{u1-u3}^{th}$')
axs[1,0].loglog(vfq,Su_t[1,0,:],label=r'$S_{u2-u1}^{th}$')
axs[1,1].loglog(vfq,Su_t[1,1,:],label=r'$S_{u2-u2}^{th}$')
axs[1,2].loglog(vfq,Su_t[1,2,:],label=r'$S_{u2-u3}^{th}$')
axs[2,0].loglog(vfq,Su_t[2,0,:],label=r'$S_{u3-u1}^{th}$')
axs[2,1].loglog(vfq,Su_t[2,1,:],label=r'$S_{u3-u2}^{th}$')
axs[2,2].loglog(vfq,Su_t[2,2,:],label=r'$S_{u3-u3}^{th}$')

for ax in axs.reshape(-1):
    ax.legend(frameon=False)
    ax.set_xlabel(r'$f [Hz]$')
    ax.set_ylabel(r'$S(f) [N^2s]$')


plt.ioff()